***Vorlesung 'Syntax natürlicher Sprachen', WS 2020/21***

---
# Übung 4 (Lösung)

In [1]:
from exercises_4 import *
from questions import aufgabe
import nltk
import sys

---

## Aufgabe 1: Eine erste (Phrasenstruktur-)Grammatik

#### Werfen Sie einen Blick auf die folgende sehr einfache kontextfreie Grammatik und erklären Sie deren Funktionsweise.

In [2]:
grammar1 = """
    S -> NP VP
    NP -> DET N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP
    V -> "gibt" | "schenkt"
"""

---
#### Sammeln Sie Sätze, die als grammatisch erkannt werden sollten, am besten in einer Liste.

In [3]:
test_sentences1 = [
    "der Mann gibt der Frau das Buch"
]

---
#### Die folgende Funktion kann Ihnen helfen, eine Reihe von Sätzen zu analysieren.

In [4]:
def test_grammar(grammar, sentences):
    cfg = nltk.CFG.fromstring(grammar)
    rd_parser = nltk.RecursiveDescentParser(cfg)
    
    for i, sent in enumerate(sentences, 1):
        print("Satz {}: {}".format(i, sent))
        results = rd_parser.parse(sent.split())
        analyzed = False
        for tree in results:
            tree.pretty_print(unicodelines=True)
            analyzed = True
        if not analyzed:
            print("Keine Analyse möglich", file=sys.stderr)

---
#### Führen Sie den Test jetzt für `grammar1` aus!

In [5]:
test_grammar(grammar1, test_sentences1)

Satz 1: der Mann gibt der Frau das Buch
                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch



---
## Aufgabe 2: Ein- und zweistellige Verben

#### Bis jetzt akzeptiert die Grammatik in `grammar1` lediglich dreistellige Verben. Erweitern Sie sie so, dass auch Verben mit weniger als zwei Objekten korrekte Verbalphrasen bilden können.

#### Folgende Sätze sollten akzeptiert werden:

In [6]:
test_sentences2 = [
    "der Mann gibt der Frau das Buch",
    "der Mann schläft",
    "das Buch gefällt der Frau",
    "die Frau kennt das Buch"
]

In [7]:
grammar2 = """
    S -> NP VP
    NP -> DET N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP | V NP | V
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
"""

In [8]:
test_grammar(grammar2, test_sentences2)

Satz 1: der Mann gibt der Frau das Buch
                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch

Satz 2: der Mann schläft
         S          
     ┌───┴──────┐    
     NP         VP  
 ┌───┴───┐      │    
DET      N      V   
 │       │      │    
der     Mann schläft

Satz 3: das Buch gefällt der Frau
                S                
     ┌──────────┴─────┐           
     │                VP         
     │          ┌─────┴───┐       
     NP         │         NP     
 ┌───┴───┐      │     ┌───┴───┐   
DET      N      V    DET      N  
 │       │      │     │       │   
das     Buch gefällt der     Frau

Satz 4: die Frau kennt das Buch
      

---

## Aufgabe 3: Beliebig lange Phrasen (rekursive X-Bar-Phrasenstruktur)

#### Erweitern Sie die Grammatik (`grammar2`) nun derart, dass Nominalphrasen auch Adjektive enthalten dürfen – und zwar beliebig viele.  Beachten Sie dabei, dass andererseits nur genau ein Determinierer in der NP-Phrase enthalten sein darf (und zwar als erstes Element). 

#### Verwenden Sie `NOM` als Label für die nominale Zwischenebene.

#### Beispiele:

In [9]:
test_sentences3 = [
    "die kluge schöne Frau kennt das Buch",
    "der schöne kluge Mann gibt der Frau das dicke Buch",
    "das dicke schöne kluge Buch schläft"
]

In [10]:
grammar3 = """
    S -> NP VP
    NP -> DET NOM 
    NOM -> ADJ NOM | N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP | V NP | V
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
"""

In [11]:
test_grammar(grammar3, test_sentences3)

Satz 1: die kluge schöne Frau kennt das Buch
                          S                     
           ┌──────────────┴──────────┐           
           NP                        │          
 ┌─────────┴────┐                    │           
 │             NOM                   VP         
 │    ┌─────────┴─────┐         ┌────┴───┐       
 │    │              NOM        │        NP     
 │    │         ┌─────┴───┐     │    ┌───┴───┐   
 │    │         │        NOM    │    │      NOM 
 │    │         │         │     │    │       │   
DET  ADJ       ADJ        N     V   DET      N  
 │    │         │         │     │    │       │   
die kluge     schöne     Frau kennt das     Buch

Satz 2: der schöne kluge Mann gibt der Frau das dicke Buch
                               S                                  
            ┌──────────────────┴────────────┐                      
            NP                              VP                    
 ┌──────────┴────┐             ┌────────┬───┴─────

---
## * Erweiterte Negativtests
#### Testen Sie auch, ob die Grammatik für die folgenden (ungrammatischen) Sätze keine Ableitung liefert:

In [12]:
negative_examples_np_rec = [
    "der der dicke der Mann gibt der Frau das Buch"
]

In [13]:
test_grammar(grammar3, negative_examples_np_rec)

Satz 1: der der dicke der Mann gibt der Frau das Buch


Keine Analyse möglich


#### Folgende Grammatik mit rekursiven NP-Regeln ohne die nominale Zwischenebene (`NOM`) erkennt dagegen diese ungrammatischen Sätze und ist insofern keine korrektes Modell der NP-Phrasenstruktur des Deutschen:

In [14]:
grammar_np_rec = """
    S -> NP VP
    NP -> N | ADJ NP | DET NP 
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP
    V -> "gibt" | "schenkt"
    ADJ -> "dicke"
"""

In [15]:
test_grammar(grammar_np_rec, negative_examples_np_rec)

Satz 1: der der dicke der Mann gibt der Frau das Buch
                           S                                 
          ┌────────────────┴─────────────────┐                
          NP                                 │               
 ┌────────┴────┐                             │                
 │             NP                            │               
 │   ┌─────────┴───┐                         │                
 │   │             NP                        VP              
 │   │    ┌────────┴───┐        ┌────────┬───┴────────┐       
 │   │    │            NP       │        NP           NP     
 │   │    │        ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
 │   │    │        │       NP   │    │       NP   │       NP 
 │   │    │        │       │    │    │       │    │       │   
DET DET  ADJ      DET      N    V   DET      N   DET      N  
 │   │    │        │       │    │    │       │    │       │   
der der dicke     der     Mann gibt der     Frau das     Buch



---
## * Hinweis zu verschiedenen Ursachen für Überproduktion von CFGs

 Die hier besprochene Begrenzung der CFG-Grammatik durch X-Bar-Regeln bezieht sich auf die Wohlgeformtheit hinsichtlich der linearen Anordnung der Wörter (als der terminalen Symbole der CFG), d. h. also auf die Konstituenten- bzw. Phrasenstruktur (deshalb auch **Phrasenstrukturgrammatik**). Dazu zählt etwa die Vermeidung der Erkennung von sich wiederholenden Determinierern (*der der Mann*) durch die Einführung einer phrasalen Zwischenebene (N' oder NOM).
 
Zu den anderen Gründe, die zu Überproduktion führen (also auch die Erkennung ungrammatischer Sätze zulassen), und die in dieser einfache CFG-Phrasenstrukturgrammatik nicht berücksichtigt sind, zählen **Subkategorisierung, Rektion und Agreement/Kongruenz**.

Diese Probleme können u.a. durch Erweiterung der Nonterminalsymbole der CFG gelöst werden, allerdings führt dies für Sprachen mit umfangreicher Morphoysntax zu einer Aufblähung der CFG-Grammatik und wird entweder über merkmalbasierte (*feature-based grammars*) oder probabilistische Erweiterungen (PCFGs) gelöst. Dazu mehr in den folgenden Sitzungen.


#### Im Folgenden sehen Sie ein Beispiel für die Nonterminal-Erweiterung einer CFG zur Lösung der Überproduktion durch die Verletzung von Subkategorisierungsbeschränkungen:

In [16]:
grammar_vp_subcat = """
    S -> NP VP
    NP -> DET N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> Vditrans NP NP | Vintrans
    Vditrans -> "gibt"
    Vintrans -> "schläft"
"""

#### Folgendene Negativbeispiele werden von dieser Grammatik nicht erkannt:

In [17]:
negative_examples_vp_subcat = [
    "der Mann schläft das Buch",
    "der Mann gibt das Buch"    
]

In [18]:
test_grammar(grammar_vp_subcat, negative_examples_vp_subcat)

Satz 1: der Mann schläft das Buch
Satz 2: der Mann gibt das Buch


Keine Analyse möglich
Keine Analyse möglich


#### Folgende Sätze (mit korrekter Anzahl an Komplementen) werden erkannt:

In [19]:
positive_examples_vp_subcat = [
    "der Mann schläft",
    "der Mann gibt der Frau das Buch"    
]

In [20]:
test_grammar(grammar_vp_subcat, positive_examples_vp_subcat)

Satz 1: der Mann schläft
         S           
     ┌───┴──────┐     
     NP         VP   
 ┌───┴───┐      │     
DET      N   Vintrans
 │       │      │     
der     Mann schläft 

Satz 2: der Mann gibt der Frau das Buch
                       S                       
     ┌─────────────────┴───────┐                
     │                         VP              
     │          ┌──────────┬───┴────────┐       
     NP         │          NP           NP     
 ┌───┴───┐      │      ┌───┴───┐    ┌───┴───┐   
DET      N   Vditrans DET      N   DET      N  
 │       │      │      │       │    │       │   
der     Mann   gibt   der     Frau das     Buch



#### Dagegen erkennt die oben erstellte `grammar2` ohne diese Berücksichtigung von Subkategorisierung auch die beiden Negativbeispiele (die die Subkategorisierungrahmen der Verben verletzen):

In [21]:
test_grammar(grammar2, negative_examples_vp_subcat)

Satz 1: der Mann schläft das Buch
                S                
     ┌──────────┴─────┐           
     │                VP         
     │          ┌─────┴───┐       
     NP         │         NP     
 ┌───┴───┐      │     ┌───┴───┐   
DET      N      V    DET      N  
 │       │      │     │       │   
der     Mann schläft das     Buch

Satz 2: der Mann gibt das Buch
              S               
     ┌────────┴────┐           
     │             VP         
     │        ┌────┴───┐       
     NP       │        NP     
 ┌───┴───┐    │    ┌───┴───┐   
DET      N    V   DET      N  
 │       │    │    │       │   
der     Mann gibt das     Buch



---
# Hausaufgaben

---
## Aufgabe 4: Eigennamen

#### Eigennamen können auch ohne Artikel eine Nominalphrase bilden. Erweitern Sie die Grammatik (`grammar3`) entsprechend.

#### Folgende Sätze sollten korrekt analysiert werden:

In [22]:
test_sentences4 = [
    "der Mann kennt Chomsky",
    "Marie gibt Fritz das Buch"
]

#### Stellen Sie außerdem sicher, dass folgende Sätze KEINE Analyse liefern!

In [23]:
negative_examples4 = [
    "Mann gibt Frau Buch",
    "Mann schläft"
    ]

In [24]:
grammar4 = """
    S -> NP VP
    NP -> DET NOM | PROPN
    NOM -> ADJ NOM | N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP | V NP | V
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
    PROPN -> "Chomsky" | "Fritz" | "Marie"
"""

In [25]:
test_grammar(grammar4, test_sentences4)

Satz 1: der Mann kennt Chomsky
              S                   
     ┌────────┴─────────┐          
     NP                 VP        
 ┌───┴───┐         ┌────┴─────┐    
 │      NOM        │          NP  
 │       │         │          │    
DET      N         V        PROPN 
 │       │         │          │    
der     Mann     kennt     Chomsky

Satz 2: Marie gibt Fritz das Buch
       S                     
  ┌────┴─────┐                
  │          VP              
  │    ┌─────┼────────┐       
  │    │     │        NP     
  │    │     │    ┌───┴───┐   
  NP   │     NP   │      NOM 
  │    │     │    │       │   
PROPN  V   PROPN DET      N  
  │    │     │    │       │   
Marie gibt Fritz das     Buch



In [26]:
test_grammar(grammar4, negative_examples4)

Satz 1: Mann gibt Frau Buch
Satz 2: Mann schläft


Keine Analyse möglich
Keine Analyse möglich


---
## Aufgabe 5: Präpositionalphrasen

#### Erweitern Sie die Grammatik (`grammar4`) nun derart, dass sowohl Nominalphrasen als auch Verbalphrasen durch Präpositionalphrasen modifiziert werden können. Erkennen Sie die Ambiguität von Übungsblatt 2 wieder?

#### Folgende Sätze sollten akzeptiert werden:

In [27]:
test_sentences5 = [
    "der Mann schläft neben dem Buch",
    "die Frau kennt das dicke Buch über Chomsky",
    "die Frau schenkt dem Mann das Buch auf dem Tisch"
]

#### Beachten Sie, dass die Form des Artikels *dem* und das Nomen *Tisch* auch noch als lexikalische Regeln ergänzt werden müssen. 

#### Stellen Sie zudem sicher, dass für den 2. und 3. Satz zwei Analysen generiert werden.

#### Achten Sie außerdem darauf, dass Ihre Erweiterung nicht die Analyse zuvor akzeptierter grammatischer Sätze verhindert!

In [28]:
grammar5 = """
    S -> NP VP
    NP -> DET NOM | PROPN | DET NOM PP
    NOM -> ADJ NOM | N
    DET -> "der" | "die" | "das" | "dem"
    N -> "Mann" | "Frau" | "Buch" | "Tisch"
    VP -> V NP NP | V NP | V | V NP NP PP | V NP PP | V PP 
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
    PROPN -> "Chomsky" | "Fritz" | "Marie"
    PP -> P NP
    P -> "neben" | "über" | "auf"
"""

In [29]:
test_grammar(grammar5, test_sentences5)

Satz 1: der Mann schläft neben dem Buch
                S                          
     ┌──────────┴───────────┐               
     │                      VP             
     │          ┌───────────┴───┐           
     │          │               PP         
     │          │      ┌────────┴───┐       
     NP         │      │            NP     
 ┌───┴───┐      │      │        ┌───┴───┐   
 │      NOM     │      │        │      NOM 
 │       │      │      │        │       │   
DET      N      V      P       DET      N  
 │       │      │      │        │       │   
der     Mann schläft neben     dem     Buch

Satz 2: die Frau kennt das dicke Buch über Chomsky
                    S                                 
     ┌──────────────┴────┐                             
     │                   VP                           
     │         ┌─────────┴────────┐                    
     │         │                  NP                  
     │         │    ┌─────────┬───┴─────────┐        

---
## * Erweiterte Lösung mit X-Bar-Phrasenstruktur

In einer X-Bar-Analyse mit Komplement-Adjunkt-Distinktion wird die PP als NP- bzw- VP-Modifizierer (=Adjunkt) dem X'-Knoten angehängt. Dies führt aber mit obiger Testfunktion zu einem `recursion error`, da der dort verwendete `rdParser` (Recursive-Descent-Parser) nicht mit linksrekursiven Regeln umgehen kann. Deshalb verwenden wir in folgender Definition einen anderen Parser (`ChartParser`).

#### Überlegen Sie, warum Satz 2 in dieser Analyse 3 Ableitungen hat.

In [30]:
def test_grammar_chart_parser(grammar, sentences):
    cfg = nltk.CFG.fromstring(grammar)
    rd_parser = nltk.ChartParser(cfg)
    
    for i, sent in enumerate(sentences, 1):
        print("Satz {}: {}".format(i, sent))
        results = rd_parser.parse(sent.split())
        analyzed = False
        for tree in results:
            tree.pretty_print(unicodelines=True)
            analyzed = True
        if not analyzed:
            print("Keine Analyse möglich", file=sys.stderr)

In [31]:
grammar5_xbar = """
    S -> NP VP
    NP -> DET NOM | PROPN
    NOM -> ADJ NOM | N
    NOM -> NOM PP
    DET -> "der" | "die" | "das" | "dem"
    N -> "Mann" | "Frau" | "Buch" | "Tisch"
    VP -> VERB
    VERB -> V NP NP | V NP | V
    VERB -> VERB PP
    V -> "gibt" | "schenkt" | "schläft" | "gefällt" | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
    PROPN -> "Chomsky" | "Fritz" | "Marie"
    PP -> P NP
    P -> "neben" | "über" | "auf"
"""

In [32]:
test_grammar_chart_parser(grammar5_xbar, test_sentences5)

Satz 1: der Mann schläft neben dem Buch
                S                           
     ┌──────────┴───────────┐                
     │                      VP              
     │                      │                
     │                     VERB             
     │          ┌───────────┴────┐           
     │          │                PP         
     │          │      ┌─────────┴───┐       
     NP         │      │             NP     
 ┌───┴───┐      │      │         ┌───┴───┐   
 │      NOM    VERB    │         │      NOM 
 │       │      │      │         │       │   
DET      N      V      P        DET      N  
 │       │      │      │         │       │   
der     Mann schläft neben      dem     Buch

Satz 2: die Frau kennt das dicke Buch über Chomsky
                    S                                      
     ┌──────────────┴─────────┐                             
     │                        VP                           
     │                        │              

---
## Aufgabe 6: Fragen zu NLTK-Kapitel 8.3

#### Lesen Sie das NLTK-Teilkapitel 8.3 (’Context Free Grammar’): http://www.nltk.org/book/ch08.html.

#### Beantworten Sie insbesondere folgende Fragen:

---
### Aufgabe 6 a:

In [33]:
aufgabe(blatt4_6a)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie ist im NLTK vorzugehen, wenn ein (wohlgef…

---
### Aufgabe 6 b:

In [34]:
aufgabe(blatt4_6b)

OpenTextfield(children=(HTML(value='<h4 style="font-size:14px;">Wie können im NLTK die Regeln einer Grammatik …

---
### Aufgabe 6 c:

#### Geben Sie 2 NP-Phrasenstrukturregeln mit indirekter Rekursion an und beantworten Sie danach die folgende Frage!

In [35]:
aufgabe(blatt4_6c)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele zeigen direkte Rekursion?</h…

---
### Aufgabe 6 d:

#### Geben Sie ein Beispiel für eine links- sowie eine rechtsrekursive NP-Phrasenstrukturregel an.

#### Beantworten Sie auch die folgenden Fragen:

In [36]:
aufgabe(blatt4_6d)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele für Phrasenstrukturregeln si…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele sind rechtsrekursiv?</h4>'),…